In [1]:
import sys
from pathlib import Path
import shutil
import json
import csv

from shapely.geometry import Point
import requests

DIR = Path('..')
sys.path.append(str(DIR))

import wavetrace as wt

%load_ext autoreload
%autoreload 2



In [2]:
# # Visually check that SRTM tiles cover NZ by 
# # making polygons and viewing them at https://geojson.io.

# tids = wt.SRTM_NZ_TILE_IDS
# collection = {
#     'type': 'FeatureCollection', 
#     'features': list(map(wt.build_feature, tids))
#     }
# json.dumps(collection)

In [3]:
# Initialize

TMP = DIR/'tmp'
wt.rm_paths(TMP)
#transmitters_path = DIR/'tests'/'data'/'transmitters.csv'
transmitters_path = DIR/'tests'/'data'/'transmitters_single.csv'
api_key = wt.get_secret('GITLAB_API_KEY') # Need this to download topo
high_definition = False
if high_definition:
    topography_path = TMP/'srtm1'
else:
    topography_path = TMP/'srtm3'


In [4]:
# Process transmitters

out_path = TMP/'splat_files'   
wt.process_transmitters(transmitters_path, out_path)

% ls {out_path}

GrayWireless_4.az  GrayWireless_4.el  GrayWireless_4.lrp  GrayWireless_4.qth


In [5]:
# Download topography

transmitters = wt.read_transmitters(transmitters_path)
print('transmitters=', transmitters)

tids = wt.compute_tile_ids(transmitters)
print('Tiles to download =', tids)

wt.download_topography(tids, topography_path, 
  high_definition=high_definition, api_key=api_key)

% ls {path}

transmitters= [{'horizontal_beamwidth': '', 'power_eirp': 4.0, 'name': 'GrayWireless_4', 'extra': '', 'antenna_height': 10.0, 'vertical_beamwidth': '', 'longitude': 174.336547, 'polarization': 0.0, 'network_name': 'Gray Wireless', 'antenna_downtilt': '', 'latitude': -35.712247, 'bearing': '0', 'frequency': 5725.0, 'site_name': '4'}]
Tiles to download = ['S36E173', 'S36E174', 'S37E173', 'S37E174']
ls: cannot access {path}: No such file or directory


In [6]:
# Process topography

out_path = TMP/'splat_files'
wt.process_topography(topography_path, out_path, 
  high_definition=high_definition)

% ls {out_path}

-36:-35:185:186.sdf  -37:-36:185:186.sdf  GrayWireless_4.az  GrayWireless_4.lrp
-36:-35:186:187.sdf  -37:-36:186:187.sdf  GrayWireless_4.el  GrayWireless_4.qth


In [7]:
# Compute coverage

in_path = TMP/'splat_files'
out_path = TMP/'coverage_reports'
wt.compute_coverage(in_path, out_path, high_definition=high_definition)

% ls {out_path}

GrayWireless_4-ck.png  GrayWireless_4.png              GrayWireless_4.tif
GrayWireless_4.kml     GrayWireless_4-site_report.txt


In [8]:
## Clean up

# wt.rm_paths(TMP)